Preprocessing in Excel:
    1. Remove rows 1-3
    2. Remove average, min, max rows at end of file
    3. Export as csv

In [1]:
import pandas as pd
import numpy as np

from datascience import *

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

In [2]:
buildingnames = ['YUAG',
                 'Berkeley',
                 'Hopper',
                 '304Elm',
                 'Davenport',
                 '38HH',
                 '320Temple',
                 '53Wall',
                 'Sprague',
                 'Malone',
                 'Trumbull',
                 '17HH']

In [3]:
# reads from csv into a new dataframe
raw = pd.read_csv('energy_raw.csv',index_col=0,na_values=['#########'])

# reindexes by appropriate datetime
raw.index = pd.to_datetime(raw.index,format='%a %m/%d/%y %H:00')

# removes built-in demand values, which tend to be bugged
raw = raw.drop(raw.columns[np.arange(0,len(buildingnames)*2,2)], axis=1)

# renames columns accordingly
raw.columns = buildingnames

In [4]:
# removes Davenport, for this example only, because its missing a few weeks of data
raw = raw.drop('Davenport',axis=1)

In [5]:
# this cell removes all consumption values BELOW first value

# creates dataframe of repeated minimum (first) values, pretty workaroundy
raw_mins = raw.copy(deep=True)
raw_mins.loc[:,:] = raw.loc['2018-01-01 00:00:00'].values

# sets all violating values to NaN
raw = raw.where(raw >= raw_mins)

# this unused line was an attempt to find outliers using std ranges
# raw = raw.where(raw > raw.median() - 2*raw.std()).where(raw < raw.median() + 2*raw.std())

raw.isnull().sum()

YUAG           3
Berkeley       3
Hopper         3
304Elm         3
38HH           3
320Temple      3
53Wall       107
Sprague        4
Malone         3
Trumbull      56
17HH         145
dtype: int64

In [6]:
# interpolates small ranges of missing consumption data (less than 12 hours)
raw = raw.interpolate(method='time',limit=6)

raw.isnull().sum()

YUAG         0
Berkeley     0
Hopper       0
304Elm       0
38HH         0
320Temple    0
53Wall       0
Sprague      0
Malone       0
Trumbull     0
17HH         0
dtype: int64

In [7]:
raw = raw.iloc[:,[0]]

raw['day'] = raw.index.day
raw['hour'] = raw.index.hour

raw_by_day = raw.resample('h').mean()

raw_by_day = raw_by_day.set_index(['day','hour']).unstack('day')

raw_by_day

ValueError: Index contains duplicate entries, cannot reshape

In [ ]:
# creates a new dataframe for the demand values, drops the first row
raw_demand = raw.diff().drop(raw.index[0])

demand = raw_demand

In [ ]:
# removes unrealistic 250% change over three hours
demand = demand.where(abs(demand.pct_change(periods=3))<2.5)

# removes strong statistical outliers
demand = demand.where(demand > 0).where(demand < demand.median()+4*demand.std())

# final interpolation of demand values after dropping outliers
# this needs to be replaced with FFT for any gaps longer than ~6 hours
demand = demand.interpolate(method='time',limit=24)

# replaces mistakenly deleted values from pct_change calculation
demand.iloc[[0,1,2],:] = raw_demand.iloc[[0,1,2],:]

print(demand.isnull().sum())

In [ ]:
# plot all demand curves to identify any remaining errors/outliers

print(demand['YUAG'].plot.line())

print(demand['Berkeley'].plot.line())

# demand['Hopper'].plot.line()

# demand['304Elm'].plot.line()

# demand['38HH'].plot.line()

# demand['320Temple'].plot.line()

# demand['53Wall'].plot.line()

# demand['Sprague'].plot.line()

# demand['Malone'].plot.line()

# demand['Trumbull'].plot.line()

# demand['17HH'].plot.line()

In [ ]:
# removes Trumbull and 17HH if I want to ignore them due to sensor errors
# final = demand.drop(['Trumbull','17HH'],axis=1)

final = final.round(1)

final.to_csv('energy_clean.csv')